In [13]:
import glob
import pandas as pd

from datetime import datetime, date

In [14]:
fp = '/home/franci/licenta/wisdm_update'
wisdm_accel_csv_files = glob.glob(fp + '/data_16*_accel_watch.csv')

In [ ]:
wisdm_accel_csv_files.sort()
print(wisdm_accel_csv_files)

In [16]:
def set_activity(activity):
    match activity:
        case 'A':
            activity = 'walking'
        case 'B':
            activity = 'jogging'
        case 'C':
            activity = 'stairs'
        case 'D':
            activity = 'sitting'
        case 'E':
            activity = 'standing'
        case 'F':
            activity = 'typing'
        case 'G':
            activity = 'brushing teeth'
        case 'H':
            activity = 'eating soup'
        case 'I':
            activity = 'eating chips'
        case 'J':
            activity = 'eating pasta'
        case 'K':
            activity = 'drinking'
        case 'L':
            activity = 'eating sandwich'
        case 'M':
            activity = 'kicking (soccer ball)'
        case 'O':
            activity = 'playing catch (tennis ball)'
        case 'P':
            activity = 'dribbling (basket ball)'
        case 'Q':
            activity = 'writing'
        case 'R':
            activity = 'clapping'
        case 'S':
            activity = 'folding clothes'
    return activity

In [17]:
def remove_semicolon(x):
    if ';' in x:
        x = x.replace(';', '')
        x = float(x)
    return x

In [18]:
def convert_to_datetime(timestamp):
    return datetime.fromtimestamp(timestamp)

In [ ]:
acol_names = ['PERSON_ID', 'ACTIVITY', "TIMESTAMP", "ACC_X", "ACC_Y", "ACC_Z"]
person_id = 101
start_timestamp = 1609459200 # Friday, January 1, 2021 12:00:00 AM | Friday, January 1, 2021 2:00:00 AM
time_step = 0.05 # 20Hz = 0.05 seconds per step
for f in wisdm_accel_csv_files:
    person_df = pd.read_csv(f, header=None, names=acol_names)

    person_df['PERSON_ID'] = person_id

    person_df['ACTIVITY'] = person_df['ACTIVITY'].apply(lambda x: set_activity(x))

    person_df['ACC_Z'] = person_df['ACC_Z'].apply(lambda x: remove_semicolon(x))

    new_timestamp = []
    current_activity = None
    counter = 0

    for id, row in person_df.iterrows():
        if row['ACTIVITY'] != current_activity:
            current_activity = row['ACTIVITY']
            counter = 0
        new_timestamp.append(start_timestamp + counter * time_step)
        counter = counter + 1
    
    person_df['TIMESTAMP'] = new_timestamp

    person_df['TIMESTAMP'] = person_df['TIMESTAMP'].apply(lambda x: convert_to_datetime(x))

    cols = cols = person_df.columns.to_list()

    activity = cols.pop(1)
    cols.append(activity)

    person_df = person_df[cols]

    if person_id == 101:
        person_df.to_csv(f, mode='w+', index=False)
        print("Saved first file with header!\n")
    else:
        person_df.to_csv(f, mode='w+', index=False, header=False)

    person_id = person_id + 1
    
print(person_id)